# ABCN2 Algorithm

The following notebook is used to import a dataset, and apply the ABCN2 algorithm to it. This uses the CN2 rule based learning algorithm, as well as expert rules which we will also derive from the dataset in this notebook

Data: Heart Attack Prediction, https://www.kaggle.com/imnikhilanand/heart-attack-prediction/downloads/heart-attack-prediction.zip/1

## Imports and reading data

In [1]:
import numpy as np
import pandas as pd
import Orange

In [2]:
FILE_INPUT = 'data.csv'

In [3]:
df = pd.read_csv(FILE_INPUT)
df = df.replace('?', np.nan)

## Data exploration

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,28,1,2,130,132,0,2,185,0,0.0,NaN,NaN,NaN,0
1,29,1,2,120,243,0,0,160,0,0.0,NaN,NaN,NaN,0
2,29,1,2,140,NaN,0,0,170,0,0.0,NaN,NaN,NaN,0
3,30,0,1,170,237,0,1,170,0,0.0,NaN,NaN,6,0
4,31,0,2,100,219,0,1,150,0,0.0,NaN,NaN,NaN,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 14 columns):
age           294 non-null int64
sex           294 non-null int64
cp            294 non-null int64
trestbps      293 non-null object
chol          271 non-null object
fbs           286 non-null object
restecg       293 non-null object
thalach       293 non-null object
exang         293 non-null object
oldpeak       294 non-null float64
slope         104 non-null object
ca            3 non-null object
thal          28 non-null object
num           294 non-null int64
dtypes: float64(1), int64(4), object(9)
memory usage: 32.3+ KB


In [6]:
df.describe()

,age,sex,cp,oldpeak,num
count,294.000000,294.000000,294.000000,294.000000,294.000000
mean,47.826531,0.724490,2.982993,0.586054,0.360544
std,7.811812,0.447533,0.965117,0.908648,0.480977
min,28.000000,0.000000,1.000000,0.000000,0.000000
25%,42.000000,0.000000,2.000000,0.000000,0.000000
50%,49.000000,1.000000,3.000000,0.000000,0.000000
75%,54.000000,1.000000,4.000000,1.000000,1.000000
max,66.000000,1.000000,4.000000,5.000000,1.000000


In [7]:
# Prints unique values
df.nunique()

age            38
sex             2
cp              4
trestbps       31
chol          153
fbs             2
restecg         3
thalach        71
exang           2
oldpeak        10
slope           3
ca              1
thal            3
num             2
dtype: int64

In [8]:
# Number of NaNs in the dataframe
df.isnull().sum()

age             0
sex             0
cp              0
trestbps        1
chol           23
fbs             8
restecg         1
thalach         1
exang           1
oldpeak         0
slope         190
ca            291
thal          266
num             0
dtype: int64

## CN2 Algorithm

In [28]:
heart_attack = Orange.data.Table(FILE_INPUT)
for x in heart_attack.domain.attributes:
    n_miss = sum(1 for d in heart_attack if np.isnan(d[x]))
    print("%4.1f%% %s" % (100.0 * n_miss / len(heart_attack), x.name))

 0.0% age
 0.0% sex
 0.0% cp
 0.3% trestbps
 7.8% chol
 2.7% fbs
 0.3% restecg
 0.3% thalach
 0.3% exang
 0.0% oldpeak
64.6% slope
99.0% ca
90.5% thal
 0.0% num       


In [25]:
# Construct a learning algorithm and classifier
cn2_classifier = Orange.classification.rules.CN2Learner(heart_attack)
cn2_classifier

# All rule-base classifiers can have their rules printed out like this:
# for r in cn2_classifier.rules:
#     print(Orange.classification.rules.rule_to_string(r))

CN2Learner(preprocessors=([28, 1, 2, 130, 132, ...],
            [29, 1, 2, 120, 243, ...],
            [29, 1, 2, 140, ?, ...],
            [30, 0, 1, 170, 237, ...],
            [31, 0, 2, 100, 219, ...],
            [32, 0, 2, 105, 198, ...],
            [32, 1, 2, 110, 225, ...],
            [32, 1, 2, 125, 254, ...],
            [33, 1, 3, 120, 298, ...],
            [34, 0, 2, 130, 161, ...],
            [34, 1, 2, 150, 214, ...],
            [34, 1, 2, 98, 220, ...],
            [35, 0, 1, 120, 160, ...],
            [35, 0, 4, 140, 167, ...],
            [35, 1, 2, 120, 308, ...],
            [35, 1, 2, 150, 264, ...],
            [36, 1, 2, 120, 166, ...],
            [36, 1, 3, 112, 340, ...],
            [36, 1, 3, 130, 209, ...],
            [36, 1, 3, 150, 160, ...],
            [37, 0, 2, 120, 260, ...],
            [37, 0, 3, 130, 211, ...],
            [37, 0, 4, 130, 173, ...],
            [37, 1, 2, 130, 283, ...],
            [37, 1, 3, 130, 194, ...],
            [3